In [ ]:
from IPython.display import HTML
HTML(open('../style.css').read())

# PicoSat

[PicoSat](https://fmv.jku.at/picosat/) is a SAT solver by [Armine Biere](https://cca.informatik.uni-freiburg.de/biere/).
A [Python API](https://github.com/conda/pycosat) to this solver is available as [pycosat](https://pypi.org/project/pycosat/).  We can install it via `pip`.

In [ ]:
pip install pycosat

In [ ]:
import pycosat

In *PicoSat* propositional variables are represented as follows:
* A propositional variable $p$ is represented as a positive natural number.
* If $n$ is the natural number representing the propositional variable $p$, then $\neg p$ is represented as the integer $-n$.
* A clause is represented as a list of integers.
* A formula in CNF is represented as a list of clauses and hence as a list of list of integers.

For example, if the propositional variables $p$, $q$, and $r$ are represented as the natural numbers
$1$, $2$, and $3$ respectively, then 
* $p \vee \neg q \vee r$ is represented as the list `[1, -2, 3]`,
* $\neg p \vee q \vee \neg r$ is represented as the list `[-1, 2, -3]`,
* $\neg p \vee \neg q \vee r$ is represented as the list `[-1, -2, 3]`, and
* $p \vee q \vee \neg r$ is represented as the list `[1, 2, -3]`.

Finally, the formula 
$$ (p \vee \neg q \vee r) \wedge (\neg p \vee q \vee \neg r) \wedge (\neg p \vee \neg q \vee r) \wedge (p \vee q \vee \neg r), $$
which is in conjunctive normal form, is represented as follows:
```
   [ [1, -2, 3], [-1, 2, -3], [-1, -2, 3], [1, 2, -3] ]
```

In [ ]:
f = [ [1, -2, 3], [-1, 2, -3], [-1, -2, 3], [1, 2, -3] ]

In order to check whether this formula is satifiable, we can use the method `pycosat.solve` as follows:

In [ ]:
pycosat.solve(f)

This shows that the formula `f` is satisfiable and that the  propositional valuation 
$$ \mathcal{I} = \{ p \mapsto \texttt{False}, q \mapsto \texttt{False}, r \mapsto \texttt{False} \} $$
is a solution for `f`, i.e. we have
$$ \mathcal{I}(\texttt{f}) = \texttt{True}. $$

## Transforming Clauses into PyCoSat Format

In order to use *PicoSat* for the examples discussed in our lecture, we need a function that transforms a formula that is in conjunctive normal form
into the format of *PicoSat*.  Furthermore, we need a function that can translate a solution found by *PicoSat* back into our format.

The function `findVariables` takes a set of `Clauses` and returns the set of all propositional variables
occurring in this set.  

In [ ]:
def findVariables(Clauses):
    Variables = set()
    for Clause in Clauses:
        for literal in Clause:
            match literal:
                case ('¬', var): Variables |= { var }
                case var       : Variables |= { var }
    return Variables

The function `numberVariables(Clauses)` takes a set of `Clauses` as input.  It returns two dictionaries:
* The dictionary `Var2Int` maps every propositional variable occurring in `Clauses` to a unique natural number.
* The dictionary `Int2Var` is the mapping that is inverse to the dictionary `Var2Int`.

In [ ]:
def numberVariables(Clauses):
    Variables = findVariables(Clauses)
    count     = 1
    Var2Int   = {}
    Int2Var   = {}
    for variable in Variables:
        Var2Int[variable] = count
        Int2Var[count   ] = variable
        count += 1
    return Var2Int, Int2Var

The function `literal2int` takes a literal and transforms this literal into an integer
representing the literal.  If the literal is a negated variable, the integer is negative, else it is positive.
`Var2Int` is a dictionary mapping propositional variables to natural numbers.

In [ ]:
def literal2int(literal, Var2Int):
    match literal:
        case ('¬', var): return -Var2Int[var]
        case var       : return  Var2Int[var]

The function `clause2pyco(Clause, Var2Int)` transforms a set of literals into a list of integers.
`Var2Int` is a dictionary mapping the propositional variables to integers.

In [ ]:
def clause2pyco(Clause, Var2Int):
    return [literal2int(literal, Var2Int) for literal in Clause]

The function `clauses2pyco(Clauses, Var2Int)` transforms a set of `Clauses` into a list of lists of integers.
`Var2Int` is a dictionary mapping the propositional variables to integers.

In [ ]:
def clauses2pyco(Clauses, Var2Int):
    return [clause2pyco(clause, Var2Int) for clause in Clauses ]

The function `int2var(Numbers, Int2Var)` takes a list of numbers representing a set of literals
and returns the associated list of literals.

In [ ]:
def int2var(Numbers, Int2Var):
    Result = set()
    for n in Numbers:
        if n > 0:
            Result |= {frozenset({Int2Var[n]})}
        else: 
            Result |= {frozenset({ ('¬', Int2Var[-n]) })}
    return Result

## Sudoku

The finish mathematician Arto Inkala claims to have created the [hardest sudoku](https://abcnews.go.com/blogs/headlines/2012/06/can-you-solve-the-hardest-ever-sudoku) ever.  It is defined below.

In [ ]:
def create_puzzle():
    return [ [ 8 , "*", "*", "*", "*", "*", "*", "*", "*"], 
             ["*", "*",  3 ,  6 , "*", "*", "*", "*", "*"],
             ["*",  7 , "*", "*",  9 , "*",  2 , "*", "*"],
             ["*",  5 , "*", "*", "*",  7 , "*", "*", "*"],
             ["*", "*", "*", "*",  4 ,  5 ,  7 , "*", "*"],
             ["*", "*", "*",  1 , "*", "*", "*",  3 , "*"],
             ["*", "*",  1 , "*", "*", "*", "*",  6 ,  8 ],
             ["*", "*",  8 ,  5 , "*", "*", "*",  1 , "*"],
             ["*",  9 , "*", "*", "*", "*",  4 , "*", "*"]
           ]

We will solve this Sudoku using the Davis-Putnam algorithm.  We use the following variables:
* `Q<r,c,d>` is a Boolean variable stating that the field in row `r` and column `c` holds the digit `d`.
  Here, `r`, `c`, `d` are all elements from the set $\{1,\cdots,9\}$.
    
The function `var(row, col, digit)` returns a formated string that is interpreted as a variable name.

In [ ]:
def var(row, col, digit):
    return f'Q<{row},{col},{digit}>'

In [ ]:
var(1,2,3)

The function `atMostOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that at most one of the variables of `S` is true.

In [ ]:
def atMostOne(S): 
    return { frozenset({('¬', p), ('¬', q)}) for p in S
                                             for q in S 
                                             if  p < q 
           }

The function `atLeastOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that at least one of the variables of `S` is true.

In [ ]:
def atLeastOne(S):
    return { frozenset(S) }

The function `exactlyOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that exactly one of the variables of `S` is true.

In [ ]:
def exactlyOne(S):
    return atMostOne(S) | atLeastOne(S)

In [ ]:
exactlyOne({'a', 'b', 'c'})

The function `allDifferent` takes a list `L` of pairs of indices as its argument.  The elements of `L` are pairs of the form
`(row, col)`, where both `row` and `col` are elements of the set $\{1, \cdots, 9\}$.
It returns a set of formulas expressing that all Sudoku fields specified by the coordinate pairs in `L` take different digits as values.

In [ ]:
def allDifferent(L):
    Clauses = set()
    for digit in range(1, 10):
        Clauses |= exactlyOne({ var(row, col, digit)  for col, row in L })
    return Clauses                                                         

In [ ]:
allDifferent([(1,c) for c in range(1,10)])

The function `exactlyOneDigit(row, col)` takes integers `row` and `col` as arguments.  These specify the row and column of a field in a Sudoku.  The function returns a set of clauses specifying that exactly one of the variables

* `Q<row,col,1>`, `Q<row,col,2>`, $\cdots$, `Q<row,col,9>`

is `True`.

In [ ]:
def exactlyOneDigit(row, col):
    return exactlyOne({ var(row, col, digit) for digit in range(1, 10) })

In [ ]:
exactlyOneDigit(1, 1)

The function `constraints_from_puzzle`  returns a set of clauses stating that the variables corresponding to numbers that are already given in the Sudoku puzzle take the values that are specified.

In [ ]:
def constraints_from_puzzle():
    Puzzle = create_puzzle()
    Variables = [ var(row + 1, col + 1, Puzzle[row][col]) for row in range(9)
                                                          for col in range(9)
                                                          if  Puzzle[row][col] != '*'
                ]
    return { frozenset({ var }) for var in Variables }

In [ ]:
constraints_from_puzzle()

The function `all_constraints` returns a CSP that encodes the given sudoku as a CSP.

In [ ]:
def all_constraints(): 
    L = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    # the constraints from the puzzle have to be satisfied
    Clauses = constraints_from_puzzle()
    # there is exactly one digit in every field
    for row in L:
        for col in L:
            Clauses |= exactlyOneDigit(row, col) 
    # all entries in a row are unique
    for row in L:
        Clauses |= allDifferent([ (row, col) for col in L ]) 
    # all entries in a column are unique
    for col in L:
        Clauses |= allDifferent([ (row, col) for row in L ])
    # all entries in a 3x3 square are unique    
    for r in range(3):
        for c in range(3):
            S = [ (r * 3 + row, c * 3 + col) for row in range(1, 4)
                                             for col in range(1, 4) 
                ]
            Clauses |= allDifferent(S)
    return Clauses

In [ ]:
for Clause in all_constraints():
    if len(Clause) == 1:
        print(set(Clause))
for Clause in all_constraints():
    if len(Clause) == 9:
        print(set(Clause))

In [ ]:
len(all_constraints())

The function `solve(Constraints, Variables)` receives two arguments:
- `Constraints` is a set of formulas representing a constraint satisfaction problem.
- `Variables`   is the set of variables that occur in this formulas.

The function computes a solution to the given problem and returns this solution.

In [ ]:
def sudoku():
    Clauses  = all_constraints()
    V2I, I2V = numberVariables(Clauses)
    Clauses  = clauses2pyco(Clauses, V2I)
    Solution = pycosat.solve(Clauses)
    if Solution != []:
        return int2var(Solution, I2V)
    else:
        print(f'The problem is not solvable!')
        return None

In [ ]:
9**3

Even though this Sudoku is modelled using $9^3 = 729$ propositional variables and we have 10551 clauses, PicoSat uses less than 40 milliseconds to solve the problem. 

In [ ]:
%%time
Solution = sudoku()

The function `remove_negative` removes the negative literals from the given solution `S` and returns the set of variables that have to be `True`.  

In [ ]:
def remove_negative(S):
    return { l for C in S for l in C if not l[0] == '¬'}

In [ ]:
Solution = remove_negative(Solution)

In [ ]:
Solution

## Graphical Representation

The following line needs to be executed once to install the package `problem_visuals`.

In [ ]:
!pip install git+https://github.com/reclinarka/problem_visuals

In [ ]:
from problem_visuals.games.sudoku.grid import Grid

In [ ]:
def arb(S):
    for x in S:
        return S

In [ ]:
def transform_solution(Solution):
    Result = {}
    for variable in Solution:
        row   = int(variable[2:3])
        col   = int(variable[4:5])
        digit = int(variable[6:7])
        Result[f'V{row}{col}'] = digit
    return Result

In [ ]:
transform_solution(Solution)

In [ ]:
def show_solution(Solution, width='50%'):
    Solution = transform_solution(Solution)
    Sudoku = create_puzzle()
    for row in range(9):
        for col in range(9):
            if Sudoku[row][col] != '*':
                del Solution[f'V{row+1}{col+1}']
    return Grid(state=Sudoku, assigned=Solution, html_width=width)

In [ ]:
show_solution(Solution)